In [1]:
import numpy as np
import pandas as pd

samsung = np.load("samsung.npy")
kospi200 = np.load("kospi200.npy")

print(samsung)
print(kospi200)

##########################################################################################
               
def split_xy(dataset, n_steps, y_column):
    x,y = list(), list()
    for i in range(len(dataset)):     
        x_end_number = i + n_steps
        y_end_number = x_end_number + y_column
        if y_end_number > len(dataset):  
            break
            
        tmp_x, tmp_y = dataset[i:x_end_number, : ], dataset[x_end_number:y_end_number, 3] 
        x.append(tmp_x)                                      
        y.append(tmp_y)
    return np.array(x), np.array(y)

x1, y1 = split_xy(samsung,5,1)
x2, y2 = split_xy(kospi200,5,1)
print(x1[0,:], '\n',y1[0])
print(x2[0,:], '\n',y2[0])

[[   53000    53900    51800    51900 39565391]
 [   52600    53200    51900    52600 23104720]
 [   52600    52800    50900    50900 16128305]
 ...
 [   59100    59700    58800    59100 16446102]
 [   58800    58800    56800    57200 20821939]
 [   57800    58400    56400    56400 19749457]]
[[   320.56    321.      316.75    316.75 173911.  ]
 [   317.79    319.53    314.95    315.87 141252.  ]
 [   315.93    316.08    312.6     314.42 123610.  ]
 ...
 [   294.38    295.67    292.45    293.98  85731.  ]
 [   293.27    294.11    287.09    288.37 101535.  ]
 [   290.24    291.47    284.53    284.53 101455.  ]]
[[   53000    53900    51800    51900 39565391]
 [   52600    53200    51900    52600 23104720]
 [   52600    52800    50900    50900 16128305]
 [   51700    51700    50600    51600 13905263]
 [   52000    52200    51200    51300 10314997]] 
 [50100]
[[   320.56    321.      316.75    316.75 173911.  ]
 [   317.79    319.53    314.95    315.87 141252.  ]
 [   315.93    316.08    

In [2]:
print(x1.shape, y1.shape, x2.shape, y2.shape)

(421, 5, 5) (421, 1) (421, 5, 5) (421, 1)


In [3]:
# 데이터셋 나누기
from sklearn.model_selection import train_test_split

x1_train, x1_test, y1_train, y1_test = train_test_split(x1,y1, 
                                                   random_state=1,
                                                    test_size=0.6,
                                                    shuffle=False)
x2_train, x2_test, y2_train, y2_test = train_test_split(x2,y2, 
                                                   random_state=1,
                                                    test_size=0.6,
                                                    shuffle=False)

In [4]:
# 데이터 전처리 StandardScaler
from sklearn.preprocessing import StandardScaler

x1_train = np.reshape(x1_train, (x1_train.shape[0], x1_train.shape[1] * x1_train.shape[2]))
x1_test = np.reshape(x1_test, (x1_test.shape[0], x1_test.shape[1] * x1_test.shape[2]))
                          
scaler = StandardScaler()
scaler.fit(x1_train)

x1_train = scaler.transform(x1_train) 
x1_test = scaler.transform(x1_test)

##########################################################################################

x2_train = np.reshape(x2_train, (x2_train.shape[0], x2_train.shape[1] * x2_train.shape[2]))
x2_test = np.reshape(x2_test, (x2_test.shape[0], x2_test.shape[1] * x2_test.shape[2]))
                          
#scaler = StandardScaler()
#scaler.fit(x2_train)

x2_train = scaler.transform(x2_train) 
x2_test = scaler.transform(x2_test)

In [5]:
print(x1_train.shape, y1_train.shape, x2_train.shape, y2_train.shape)

(168, 25) (168, 1) (168, 25) (168, 1)


In [6]:
# 데이터 다시 Reshape (DNN 들어갈 모델 / LSTM 들어갈 모델)
x1_train = np.reshape(x1_train, (x1_train.shape[0], 5, 5))
x1_test = np.reshape(x1_test, (x1_test.shape[0], 5, 5))

##########################################################################################

x2_train = np.reshape(x2_train, (x2_train.shape[0], 5, 5))
x2_test = np.reshape(x2_test, (x2_test.shape[0], 5, 5))

In [7]:
#2. 모델 구성
from keras.models import Model
from keras import layers

input_1 = layers.Input(shape=(5,5))
input_tensor_1 = layers.LSTM(32, activation='tanh')(input_1)   # x1에 투입할 모델
hidden_layer_1 = layers.Dense(16, activation='relu')(input_tensor_1)
hidden_layer_1 = layers.Dense(8, activation='relu')(hidden_layer_1)
output_tensor_1 = layers.Dense(1)(hidden_layer_1)

input_2 = layers.Input(shape=(5,5))
input_tensor_2 = layers.LSTM(32, activation='tanh')(input_2)    # x2에 투입할 모델
hidden_layer_2 = layers.Dense(16, activation='relu')(input_tensor_2)
hidden_layer_2 = layers.Dense(8, activation='relu')(hidden_layer_2)
output_tensor_2 = layers.Dense(1)(hidden_layer_2)

from keras.layers.merge import concatenate , Add

merged_model = concatenate(inputs=[output_tensor_1, output_tensor_2])  # 두 개 이상은 []로 묶기
# merged_model = Add()([output_tensor_1, output_tensor_2])  # concatenate 대신 Add를 사용해도 가능하다.

output_tensor_3 = layers.Dense(8)(merged_model)        # 첫 번째 아웃풋 모델
output_tensor_3 = layers.Dense(1)(output_tensor_3)


model = Model(inputs=[input_1, input_2],
              outputs=output_tensor_3)

model.summary()

Using TensorFlow backend.
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\d

Instructions for updating:
Colocations handled automatically by placer.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 5, 5)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 5, 5)         0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           4864        input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 32)           4864        input_2[0][0]                    
____________________

In [8]:
#3. 훈련
model.compile(loss='mse', optimizer='adam', metrics=['mae'])
model.fit([x1_train, x2_train], y1_train, epochs=100, batch_size=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
168/168 [==============================] - 1s 7ms/step - loss: 2021649946.6667 - mae: 44836.9219
Epoch 2/100
168/168 [==============================] - 0s 889us/step - loss: 2021338818.2857 - mae: 44833.4570
Epoch 3/100
168/168 [==============================] - 0s 928us/step - loss: 2020168038.8571 - mae: 44820.4062
Epoch 4/100
168/168 [==============================] - 0s 827us/step - loss: 2016600521.1429 - mae: 44780.6992
Epoch 5/100
168/168 [==============================] - 0s 873us/step - loss: 2006664112.7619 - mae: 44669.5820
Epoch 6/100
168/168 [==============================] - 0s 872us/step - loss: 1982012678.0952 - mae: 44393.7695
Epoch 7/100
168/168 [==============================] - 0s 871us/step - loss: 1935425177.1429 - mae: 43865.2578
Epoch 8/100
168/168 [==============================] - 0s 863us/step - loss: 1854425938.2857 - mae: 42933.4414
Epoch 9/100
168/168 [==============================] - 0s 797us/st

168/168 [==============================] - 0s 772us/step - loss: 657610.9283 - mae: 638.1492
Epoch 75/100
168/168 [==============================] - 0s 790us/step - loss: 652746.8643 - mae: 640.7409
Epoch 76/100
168/168 [==============================] - 0s 790us/step - loss: 642131.6582 - mae: 627.9054
Epoch 77/100
168/168 [==============================] - 0s 784us/step - loss: 649655.0180 - mae: 634.3304
Epoch 78/100
168/168 [==============================] - 0s 795us/step - loss: 651232.6185 - mae: 647.3983
Epoch 79/100
168/168 [==============================] - 0s 790us/step - loss: 655411.9464 - mae: 626.9874
Epoch 80/100
168/168 [==============================] - 0s 772us/step - loss: 626489.8661 - mae: 621.3532
Epoch 81/100
168/168 [==============================] - 0s 790us/step - loss: 658872.7142 - mae: 642.0807
Epoch 82/100
168/168 [==============================] - 0s 772us/step - loss: 639299.9054 - mae: 636.3892
Epoch 83/100
168/168 [==============================] - 0s 

In [9]:
results_acc = model.evaluate([x1_test, x2_test], y1_test, batch_size=5)
print('results_acc: ', results_acc)

253/253 [==============================] - 0s 910us/step
results_acc:  [6087304.974987648, 1353.8358154296875]


In [20]:
# RMSE 만들기
from sklearn.metrics import mean_squared_error
y_predict = model.predict([x1_test,x2_test], batch_size=1)

def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE :", RMSE(y1_test, y_predict))

# R2 지표 만들기
from sklearn.metrics import r2_score

r2_y_predict = r2_score(y1_test, y_predict)
print("R2 : ", r2_y_predict)

RMSE : 2467.246811286235
R2 :  0.7212017095123235


In [21]:
for i in range(5):
    print("종가(test) : {}".format(y1_test[i]))
    print("종가(pred) : {}".format(y_predict[i]))

종가(test) : [42750]
종가(pred) : [41994.188]
종가(test) : [42150]
종가(pred) : [42390.527]
종가(test) : [42000]
종가(pred) : [42416.77]
종가(test) : [43050]
종가(pred) : [42058.043]
종가(test) : [44750]
종가(pred) : [42594.715]
